<a href="https://colab.research.google.com/github/lorek/MethodsClassDimRed/blob/main/MoCaDR_List_nr_15_MCMC_Metropolis_decryption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NOTEBOOK NO 15: Markov Chain Monte Carlo
## Metropolis algorithm: decrypting substitution cipher

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import copy as cp
import pandas as pd
import inspect
import urllib.request
import re
from collections import Counter
import string;




In [2]:
#ttest

## Read in cryptogram and book. Clean data

In [3]:

cryptogram_data=urllib.request.urlopen("https://raw.githubusercontent.com/lorek/datasets/master/encrypted_message.txt")
for line in cryptogram_data:
  cryptogram=line.decode('utf-8')



In [4]:
print(cryptogram[:100])
print(cryptogram[-100:])

print(len(cryptogram))


sjhmsXLmKNKmNomsXLmIpwpPjKmoXmKjeBgComoQNhmIjhhpPjmnFjphjmaBNojmjIpNFmoXmoQjmFjeoLBjBmaNoQmhFXPpwmGj
omAXIjmlwmqpImijgmbXamGXXwmEXmDMjwmnpCjBmOLjjwmvXXTmzjMjwmdBpMjFmYwNMjBhjmfNKjXmJXXFm BpgmsjhmujkBp

10016


Read Tolsto's "War and Peace"

In [5]:
book_data=urllib.request.urlopen("https://raw.githubusercontent.com/lorek/datasets/master/war_and_peace.txt")
book=""
for line in book_data:
  book=book+str(line.decode('utf-8'))
print(len(book))

3238412


In [6]:
print(book[0:220])





The Project Gutenberg EBook of War and Peace, by Leo Tolstoy

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the t


Clean data, make sure only a-z, A-Z and space are present

In [7]:
#clean

regex = re.compile('[^a-zA-Z ]')

regex_space=re.compile('\n')
book=re.sub('\s+', ' ', book).strip()
book=regex_space.sub(' ',book)
book=regex.sub('',book).strip()
print(len(book))
book=book.rstrip('\n')
print(len(book))

print(book[0:220])

cryptogram=re.sub('\s+', ' ', cryptogram).strip()
cryptogram=regex_space.sub(' ',cryptogram)
cryptogram=regex.sub('',cryptogram)
print(len(cryptogram))
cryptogram=cryptogram.rstrip('\n')
print(len(cryptogram))

print(cryptogram[:100])
print(cryptogram[-100:])
print(cryptogram)



3087852
3087852
The Project Gutenberg EBook of War and Peace by Leo Tolstoy This eBook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever You may copy it give it away or reuse it under the terms of t
10015
10015
sjhmsXLmKNKmNomsXLmIpwpPjKmoXmKjeBgComoQNhmIjhhpPjmnFjphjmaBNojmjIpNFmoXmoQjmFjeoLBjBmaNoQmhFXPpwmGj
homAXIjmlwmqpImijgmbXamGXXwmEXmDMjwmnpCjBmOLjjwmvXXTmzjMjwmdBpMjFmYwNMjBhjmfNKjXmJXXFm BpgmsjhmujkBp
sjhmsXLmKNKmNomsXLmIpwpPjKmoXmKjeBgComoQNhmIjhhpPjmnFjphjmaBNojmjIpNFmoXmoQjmFjeoLBjBmaNoQmhFXPpwmGjoBXCXFNhmpFPXBNoQImpFFXahmoXmkBjpWmhLkhoNoLoNXwmeNCQjBhmrjheBNkjmhQXBoFgmgXLBmpCCBXpeQmdXmIpWjmNomjphNjBmoXmKjeBgComlmCLomQjBjmIXBjmhjwojwejhmNomKXjhmIpWjmNomjphNjBmTXBmGjoBXCXFNhmoXmKjeBgComNomlwmCpBoNeLFpBmNomNhmjphNjBmNTmpFFmoQjmFXajBmpwKmLCCjBmFjoojBhmpCCjpBmNwmoQNhmIjhhpPjmVjFXamNwTXBIpoNXwmXwmGjoBXCXFNhmpFPXBNoQImTBXImJNWNCjKNpmNhmeXCNjKmdQjmpFPXBNoQImaphmwpIjKmpTojBmENeQXFphmGjoBXCXFNhmaQXmpLoQXBjKmoQjmmpBoNeFjmycLpoNXwmXTmzopojmtpFeLFpoN

## Compute frequencies $M$

In [8]:
from collections import Counter
import string

In [9]:
letters = list(book)
letters_unique = sorted(set(letters + list(string.ascii_letters) + list(" ")))
print(letters_unique)

# Compute bigram frequencies
cntr = Counter(zip(letters, letters[1:]))
total = sum(cntr.values())

# Create the initial frequency dictionaries
M2 = {''.join(k): v for k, v in cntr.items()}

# Normalize the frequencies to get M
bigram_sums = {a: sum(M2.get(a+b, 0) for b in letters_unique) for a in letters_unique}
M = {k: v / bigram_sums[k[0]] for k, v in M2.items()}

print(M)
print(M2)
print("\n")
print("M['aa'] = ", M['aa']);
print("M['st'] = ", M['st'])

[' ', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
{'Th': 0.7169257846875917, 'he': 0.4464671246319922, 'e ': 0.35629713859872286, ' P': 0.010167504246760632, 'Pr': 0.3698588090853284, 'ro': 0.09330352536543422, 'oj': 0.0006208543380206951, 'je': 0.22992056487202117, 'ec': 0.01744529674028961, 'ct': 0.06930160650668818, 't ': 0.2651450638226173, ' G': 0.0022478695564651413, 'Gu': 0.14581734458940906, 'ut': 0.17064117160815384, 'te': 0.08347260613953997, 'en': 0.08384085623610736, 'nb': 0.0008473121376079215, 'be': 0.3447231973205372, 'er': 0.13908376055197935, 'rg': 0.010269991401547721, 'g ': 0.44280745242283703, ' E': 0.002383836528851485, 'EB': 0.002215330084182543, 'Bo': 0.273987798114254, 'oo': 0.038079066065269306, 'ok': 0.017718227646590607, 'k ': 0.26124486506161926,

## Mapping letters <-> numbers

In [10]:
characters = list(dict.fromkeys(letters))
characters = list(dict.fromkeys(letters_unique))

number_of_characters=len(characters)

#mapping: letter -> number
char_mapping = dict(zip(letters_unique ,np.arange(len(characters))))
#print(char_mapping)

#inverse mapping: number -> letter
char_mapping_inv = dict(map(reversed, char_mapping.items()))

In [11]:
print(char_mapping)
print(char_mapping_inv)


{' ': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'J': 10, 'K': 11, 'L': 12, 'M': 13, 'N': 14, 'O': 15, 'P': 16, 'Q': 17, 'R': 18, 'S': 19, 'T': 20, 'U': 21, 'V': 22, 'W': 23, 'X': 24, 'Y': 25, 'Z': 26, 'a': 27, 'b': 28, 'c': 29, 'd': 30, 'e': 31, 'f': 32, 'g': 33, 'h': 34, 'i': 35, 'j': 36, 'k': 37, 'l': 38, 'm': 39, 'n': 40, 'o': 41, 'p': 42, 'q': 43, 'r': 44, 's': 45, 't': 46, 'u': 47, 'v': 48, 'w': 49, 'x': 50, 'y': 51, 'z': 52}
{0: ' ', 1: 'A', 2: 'B', 3: 'C', 4: 'D', 5: 'E', 6: 'F', 7: 'G', 8: 'H', 9: 'I', 10: 'J', 11: 'K', 12: 'L', 13: 'M', 14: 'N', 15: 'O', 16: 'P', 17: 'Q', 18: 'R', 19: 'S', 20: 'T', 21: 'U', 22: 'V', 23: 'W', 24: 'X', 25: 'Y', 26: 'Z', 27: 'a', 28: 'b', 29: 'c', 30: 'd', 31: 'e', 32: 'f', 33: 'g', 34: 'h', 35: 'i', 36: 'j', 37: 'k', 38: 'l', 39: 'm', 40: 'n', 41: 'o', 42: 'p', 43: 'q', 44: 'r', 45: 's', 46: 't', 47: 'u', 48: 'v', 49: 'w', 50: 'x', 51: 'y', 52: 'z'}


## Main function: calc_likelihood_ratio
It computes:
$${f(\sigma')\over f(\sigma)}=\prod_{i=1}^{L-1}{\textbf{M}(\sigma(c_i),\sigma(c_{i+1}))
\over \textbf{M}(\sigma'(c_i),\sigma'(c_{i+1}))},$$

In [12]:
def calc_likelihood_ratio(encrypted_message, M, sigma, i, j, char_mapping_inv, char_mapping):
  # compute f(sigma')/f(sigma)
  # do not compute full f(sigma') and f(sigma), to large (small) numbers
  # Moreover, add some penalty if 'xy' is not in decrypted text

    likelihood_ratio = 1;
    sigma2=cp.copy(sigma)
    sigma2[[i,j]]=sigma2[[j,i]]

    M_min = min(M.values())
    penalty = M_min/10;

    for k in np.arange(len(encrypted_message)-1):
        a0=encrypted_message[k]
        b0=encrypted_message[k+1]
        if (char_mapping[a0]==i or char_mapping[a0]==j or char_mapping[b0]==i or char_mapping[b0]==j):

            a=char_mapping_inv[sigma[char_mapping[a0]]]
            b=char_mapping_inv[sigma[char_mapping[b0]]]
            ab=a+b

            a2=char_mapping_inv[sigma2[char_mapping[a0]]]
            b2=char_mapping_inv[sigma2[char_mapping[b0]]]
            ab2=a2+b2

            #print("ASDF ab = ", ab, ", ab2 = ", ab2)
            #if ((ab in M) and (ab2 in M)):
            if(ab2 in M):
                likelihood_ratio = likelihood_ratio*M[ab2];
            else:
                likelihood_ratio = likelihood_ratio*penalty;

            if(ab in M):
                likelihood_ratio = likelihood_ratio/M[ab];
            else:
                likelihood_ratio = likelihood_ratio/penalty;


                #print(" i = ", i,", j = ", j, ", a0 = ", a0, ", b0 = ", b0, ", k = ", k, "lik_rat=", likelihood_ratio)


            #quit()

    return likelihood_ratio;

In [13]:
def transform_message(text_message, perm, char_mapping, char_mapping_inv):
    decrypted_message = ""
    for let in text_message:
        decrypted_message = decrypted_message +char_mapping_inv[perm[char_mapping[let]]]
    return decrypted_message


## The Metropolis Algorithm
To sample from a distribution $\pi(\textbf{w})$, repeat *enough* number of steps (recall, we assume each $\textbf{w}$ has $m$ neighbours $\mathcal{N}(\mathbf{w})$:


*   Assume we are currently $\textbf{w}^{(t)}$
*   Choose a neighbour $\textbf{w}'$ of $\textbf{w}^{(t)}$ uniformly at random (i.e., with prob. $1/m$)
*   Set $\textbf{w}^{(t+1)}=\textbf{w}'$ with probability $\alpha=\min\left(1,{\pi(\textbf{w}')\over \pi(\textbf{w}^{(t)})}\right)$;
otherwise $\textbf{w}^{(t+1)}=\textbf{w}^{(t)}$ (i.e., do nothing)




In [14]:
R=10000
#TRY ANOTHER SEED !!
np.random.seed(1434)
print(np.random.permutation(53))
print(number_of_characters)
sigma=np.random.permutation(number_of_characters)

tenth=int(R/10)
show_length=50
# print(sigma)
# print(M['ab']); print(M['st'])
# print(char_mapping)
# print(char_mapping_inv[char_mapping['d']])
# print(len(book))

[25 39 47 31 40 44 27  4 30 23 11 28  3 52 24  9 17  6 22 49 10 12 43 51
 45  2 42 26 37 35 32 20 18  0  8 29 34  7 15 16 13 14 38 19 46 21  5 33
 36 50 41  1 48]
53


In [15]:
print(M)

{'Th': 0.7169257846875917, 'he': 0.4464671246319922, 'e ': 0.35629713859872286, ' P': 0.010167504246760632, 'Pr': 0.3698588090853284, 'ro': 0.09330352536543422, 'oj': 0.0006208543380206951, 'je': 0.22992056487202117, 'ec': 0.01744529674028961, 'ct': 0.06930160650668818, 't ': 0.2651450638226173, ' G': 0.0022478695564651413, 'Gu': 0.14581734458940906, 'ut': 0.17064117160815384, 'te': 0.08347260613953997, 'en': 0.08384085623610736, 'nb': 0.0008473121376079215, 'be': 0.3447231973205372, 'er': 0.13908376055197935, 'rg': 0.010269991401547721, 'g ': 0.44280745242283703, ' E': 0.002383836528851485, 'EB': 0.002215330084182543, 'Bo': 0.273987798114254, 'oo': 0.038079066065269306, 'ok': 0.017718227646590607, 'k ': 0.26124486506161926, ' o': 0.05749990288073401, 'of': 0.08932342796497744, 'f ': 0.3659591343430147, ' W': 0.005080220513707943, 'Wa': 0.03393351800554017, 'ar': 0.08904092130283396, 'r ': 0.2122304385210662, ' a': 0.11252767899080722, 'an': 0.21658671370381888, 'nd': 0.191464853160252

In [16]:

for r in np.arange(R):

  if (np.mod(r,tenth)==0 or r<10):
    metropolis_decrypted = transform_message(cryptogram, sigma, char_mapping, char_mapping_inv)
    print("r = ", r, "=",np.round(100*r/R,1),"%\tdecrypted = ", metropolis_decrypted[:show_length])

  sigma2=cp.copy(sigma)

  #random perm, but only take first two elements = i,j uniform and distinct
  tmp=np.random.permutation(number_of_characters)
  i=tmp[0]
  j=tmp[1]
  sigma2[[i,j]]=sigma2[[j,i]]

  ratio2 = calc_likelihood_ratio(cryptogram, M, sigma, i, j, char_mapping_inv, char_mapping)

  alpha=min(1,ratio2)
  U=np.random.random(1)[0]
  if (U<=alpha):
      sigma=cp.copy(sigma2)

r =  0 = 0.0 %	decrypted =  mq RmKgRyGyRGIRmKgRMFvFfqyRIKRyqPtNzIRIsG RMq  Ffq
r =  1 = 0.0 %	decrypted =  mq RmKgRyGyRGIRmKgRMFvFfqyRIKRyqPtNzIRIsG RMq  Ffq
r =  2 = 0.0 %	decrypted =  mq RmKgRyGyRGIRmKgRMFvFfqyRIKRyqPtNzIRIsG RMq  Ffq
r =  3 = 0.0 %	decrypted =  mq RmKgRyGyRGIRmKgRMFvFfqyRIKRyqPtNzIRIsG RMq  Ffq
r =  4 = 0.0 %	decrypted =  mq RmKgRyGyRGIRmKgRMFvFfqyRIKRyqPtNzIRIsG RMq  Ffq
r =  5 = 0.0 %	decrypted =  mq RmKgRyGyRGIRmKgRMFvFfqyRIKRyqPtNzIRIsG RMq  Ffq
r =  6 = 0.1 %	decrypted =  mq RmOgRyGyRGIRmOgRMFvFfqyRIORyqPtNzIRIsG RMq  Ffq
r =  7 = 0.1 %	decrypted =  mq RmOgRyGyRGIRmOgRMFvFfqyRIORyqPtNzIRIsG RMq  Ffq
r =  8 = 0.1 %	decrypted =  mq RmOgRyGyRGIRmOgRMFvFfqyRIORyqntNzIRIsG RMq  Ffq
r =  9 = 0.1 %	decrypted =  mq RmOgRyGyRGIRmOgRMFvFfqyRIORyqntNzIRIsG RMq  Ffq
r =  1000 = 10.0 %	decrypted =  Mi eMmhenaneareMmheutstkinermeniyofprerda eui  tki
r =  2000 = 20.0 %	decrypted =  LimeLadenoneo eLadeutrtkine aenicsgp e homeuimmtki
r =  3000 = 30.0 %	decrypted =  ZimeZayenone

In [17]:
print(sigma)

[26 14 44 42 21 12 38  7 50 39 11 30 47 48 35 17 33 34  4  1 32 52  2 37
 41 15 36 49  6 43 20 29 22 51 45 24 31 28  9  0  3 46 27 10 16 25 18 23
 13 40  8  5 19]


In [18]:
metropolis_decrypted = transform_message(cryptogram, sigma, char_mapping, char_mapping_inv)
print(metropolis_decrypted)

Yes You did it You managed to decrypt this message Clease write email to the lecturer with slogan Getropolis algorithm allows to break substitution ciphers Pescribe shortly your approach To make it easier to decrypt I put here more sentences it does make it easier for Getropolis to decrypt it In particular it is easier if all the lower and upper letters appear in this message Below information on Getropolis algorithm from Kikipedia is copied The algorithm was named after Licholas Getropolis who authored the  article Equation of State Ralculations by Dast Romputing Gachines together with Arianna K Mosenbluth Garshall Mosenbluth Augusta N Teller and Edward Teller This article proposed the algorithm for the case of symmetrical proposal distributions and K X Nastings extended it to the more general case in  Some controversy exists with regard to credit for development of the algorithm Getropolis had coined the term Gonte Rarlo in an earlier article with Stanislav Olam was familiar with the

# Q15.1
* Compute frequency matrix M from another English text and repeat the procedure: encrypt and then decrypt message of your choice. Try shorter and longer messages.